In [1]:
from transformers import DonutProcessor
import sys
sys.path.insert(1, 'PubTabNet/src/')
sys.path.insert(1, '../proj/src/')

from metric import TEDS
from processing_tabeleiro import TabeleiroProcessor
import json
import os


SPLIT = "train/"

PUBTABNET = 'data/anns/'

ANN_PATH = PUBTABNET

json_list = os.listdir(ANN_PATH + SPLIT)

aux_list = []
html_list = []

for f in json_list:
    if(f[-6:-5] == "L"):
        html_list.append(f)
    else:
        aux_list.append(f)
json_list = aux_list
processor = TabeleiroProcessor.from_pretrained("processors/donut-base")

/home/jao/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
from transformers import AddedToken

new_tokens = ["<table_extraction>", "<table>", "<row>", "<cell>", "<row_and_col_header>", "<row_header>", "<col_header>"]
new_tokens += ["<content_row_and_col_header>", "<content_row_header>", "<content_col_header>", "<content>"]

for i in range(2):
    for j in range(2):
        for k in range(2):
            new_tokens.append("<span_type=0" + str(i) + str(j) + str(k) + ">")
            new_tokens.append("<span_type=1" + str(i) + str(j) + str(k) + ">")

processor.tokenizer.add_tokens(new_tokens, special_tokens = False)

27

In [3]:
json_list[:10]

['PMC3549936_002_00.json',
 'PMC3112512_009_00.json',
 'PMC3772909_007_00.json',
 'PMC5294720_004_00.json',
 'PMC3446317_008_00.json',
 'PMC5071235_005_00.json',
 'PMC1421390_001_00.json',
 'PMC5251219_004_01.json',
 'PMC3973966_003_00.json',
 'PMC4147206_002_00.json']

In [4]:
gt_list = []
pred_list = []

teds = TEDS(n_jobs=4, structure_only = False)

In [5]:
processor.tokenizer("</s>")

{'input_ids': [0, 2, 2], 'attention_mask': [1, 1, 1]}

In [ ]:
import random
import torch
from difflib import SequenceMatcher
from tqdm.auto import tqdm

removed_tags = []
soma_ratios = 0

for i, file_name in enumerate(tqdm(json_list)):

    with open(ANN_PATH + SPLIT + file_name, encoding="utf-8") as f:
        annotation = json.load(f)
    with open(ANN_PATH + SPLIT + file_name[:-5]+"-HTML.json", encoding="utf-8") as f:
        gt_html = "<html><body><table>"+str(json.load(f))+"</table></body></html>"
    
    for tag in removed_tags:
        gt_html = "<html><body><table>"+gt_html.replace(tag, "")+"</table></body></html>"
    
    target_sequence = "<s>"+processor.json2token(annotation)+"</s>"
    input_ids = processor.tokenizer(
        target_sequence,
        add_special_tokens=False,
        max_length= 4096,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
    )["input_ids"].squeeze(0)
    
    
    input_ids = torch.cat((input_ids, torch.Tensor([2, 2]).int()), 0)
    
    table = processor.token2ann( input_ids, 1)
    table_aux  = table.copy()
    
    
    pred_html = "<html><body><table>"+processor.table2html(table['tables'][0])+"</table></body></html>"
    
    pred_html = pred_html
    gt_html = gt_html
    
    pred_list.append(pred_html)
    gt_list.append(gt_html)
    
    ratio = teds.evaluate(str(pred_html), str(gt_html))
    print(ratio)
    if(ratio < .5):
        print(processor.decode(input_ids[1]))
        #print(target_sequence)
        #print(table['tables'][0])
        print(pred_html)
        print()
        print(gt_html)
        print(ratio)
    if i% 10 == 0:   
        print(str(i)+'->', soma_ratios/max(1, i))
    
    soma_ratios += ratio
print(soma_ratios/len(json_list))

  0%|                                    | 1/500777 [00:03<452:41:09,  3.25s/it]

0.661344697917104
0-> 0.0


  0%|                                    | 2/500777 [00:04<260:53:26,  1.88s/it]

0.9174242424242425


  0%|                                    | 3/500777 [00:07<379:27:43,  2.73s/it]

...

1.0


  0%|                                  | 355/500777 [06:40<134:01:24,  1.04it/s]

0.7526569885083817
0.9032258064516129


  0%|                                  | 357/500777 [06:42<114:30:29,  1.21it/s]

1.0


In [ ]:
input_ids

In [ ]:
json.dump(pred_list, open("teste-pred.json", 'w'))
json.dump(gt_list, open("teste-gt.json", 'w'))

In [ ]:
a = '<thead><tr><td rowspan="2"><b>Cutoff (IRS)</b></td><td rowspan="2"><b>Number</b></td><td colspan="2"><b>Overall survival prognosis</b></td></tr><tr><td><b>Log-rank <i>P</i></b></td><td><b>Corrected <i>P</i><sup><i>∗</i></sup></b></td></tr></thead><tbody><tr><td>≥1</td><td>99/94</td><td>5.56</td><td>0.0184</td></tr><tr><td>≥2</td><td>129/64</td><td>4.37</td><td>0.0377</td></tr><tr><td>≥3</td><td>150/43</td><td>8.37</td><td>0.0038</td></tr><tr><td>≥4</td><td>161/32</td><td>6.35</td><td>0.0117</td></tr><tr><td>≥5 </td><td>177/16</td><td>4.50</td><td>0.0338</td></tr></tbody>'

for tag in ["<i>", "</i>","<b>","</b>"]:
    a = a.replace(tag, "")
    
a

In [ ]:
target_sequence = "Capítulo Primeiro Do título Uma noite destas, vindo da cidade para o Engenho Novo, encontrei no[ 1 ] trem da Central um rapaz aqui do bairro, que eu conheço de vista e de cha- péu. Comprimentou-me, sentou-se ao pé de mim, falou da lua e dos mi- nistros, e acabou recitando-me versos. A viagem era curta, e os versos pode ser que não fossem inteiramente maus. Succedeu, porém, que, como eu es- tava cansado, fechei os olhos três ou quatro vezes; tanto bastou para que ele interrompesse a leitura e metesse os versos no bolso. — Continue, disse eu acordando.[ 2 ] — Já acabei, murmurou ele.[ 3 ] — São muito bonitos.[ 4 ] Vi-lhe fazer um gesto para tirá-los outra vez do bolso, mas não passou[ 5 ] do gesto; estava amuado. No dia seguinte entrou a dizer de mim nomes feios, e acabou alcunhando-me Dom Casmurro."

input_ids = processor.tokenizer(
        target_sequence,
        add_special_tokens=True,
        max_length= 1536,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
    )["input_ids"].squeeze(0)

In [ ]:
input_ids

In [ ]:
for token in input_ids:
    print(processor.decode(token))

In [ ]:
processor.decode(input_ids)